# Neutral Atom Mapper Visualizer

## Generation of CSV File

First, a CSV file is generated that defines the location of every atom at each time step. Additionally, different colors and sizes of atoms can be used to depict the application of various quantum operations.

### Input
Takes a description of a circuit compiled for the neutral atoms platform. The input must follow the following format:
- Line comments start with `#`, those lines are ignored during parsing.
- allowed keywords are `init`, `move`, `rx`, `ry`, `cz`, `czz`
    - theres is one keyword per line and the line starts with the keyword
    - `init` must not follow after any other keyword
    - the keywords `ry` and `ry` receive a parameter, e.g. `ry(pi/2)`
    - every keyword is followed by a sequence of natural numbers defining the coordinates of atoms, e.g. `init 6 24 12 24` means that one atom is placed at $(6,24)$ and one at $(12,24)$. Here, the unit µm is assumed. For a move the syntax is $\mathtt{move\ (x_{old}\ y_{old}\ x_{new}\ y_{new})^+}$.

### Output
A CSV file containing the postion of every atom at each point in time. The file has the following format

time_frame | atom | xpos | ypos | gate | size
---|---|---|---|---|---
0 | 0 | 6 | 24 | 1 | 1
0 | 1 | 12 | 24 | 1 | 1
0 | 2 | 18 | 24 | 1 | 1
1 | 0 | 6 | 24 | 2 | 2
1 | 1 | 12 | 24 | 2 | 2
1 | 2 | 18 | 24 | 2 | 2
... | ... | ... | ... | ... | ...

<br>

> **_NOTE_**
> 
> It is important that the location of every atom is specified at every time step even though no operation is performed on an atom.



In [1]:
from re import match, findall  # use of regex for parsing input
from copy import deepcopy  # make a deep copy dicts, ...

The following snippet creates the data that specifies the grid of SLM tweazers where atoms can be placed. It is based on the description in [1, p. 24].

1. D. Bluvstein et al., ‘Logical quantum processor based on reconfigurable atom arrays’, Nature, Dec. 2023, doi: [10.1038/s41586-023-06927-3](https://doi.org/10.1038/s41586-023-06927-3).

In [2]:
with open("sites.csv", "w") as file:
    file.write("xpos;ypos;zone;size\n")
    for y in range(20):
        for x in range(20):
            if y < 4: # entangling zone
                # sites are shifted horizontally by a minor offset
                # such that neighboring atoms are within interaction radius
                if x % 2 == 0: # smal shift to the right
                    file.write(f"{x*6+2};{y*6};entangling;1\n")
                else: # small shift to the left
                    file.write(f"{x*6-2};{y*6};entangling;1\n")
            elif y < 16: # storage zone
                file.write(f"{x*6};{y*6+20};storage;1\n")
            else: # readout zone
                file.write(f"{x*6};{y*6+40};readout;1\n")

In [3]:
class ParseError(Exception):
    '''
    Custom Exception that is raised when a parsing error occurs,
    e.g. due to an syntax error.
    '''
    def __init__(self, msg):
        super().__init__(msg)

In [4]:
def get_coords(line):
    '''
    This function takes a string of space separated natural numbers.
    There must be even many of those numbers.
    It returns a list of tuples, representing x- and y-coordinates.

        Parameters:
            line (str): string of space separated integers

        Returns:
            a list of tuples of ints
    '''
    return [tuple(map(int, findall("\d+", p))) for p in findall(r"\d+ \d+", line)]

The following cell reads the circuit description from 'circuit.txt' and produces the CSV file described above.

In [5]:
init = True     # true iff no other keyword except init occured yet
pos2atoms = {}  # a mapping from position to atom id, i.e. it is a 2D dictionary
atoms2pos = []  # a list that maps atom ids to positions (tuuples of ints)
atoms_n = 0     # number of atoms, used to determine the id of an atom

with open("circuit.txt", "r") as src: # open the output file
    with open("data.csv", "w") as csv: # open the input file
        csv.write("time_frame;atom;xpos;ypos;gate;size\n") # write header
        lineno = 0 # linenumber in the input file, correponds to timesteps (comments are ignored)
        while True:
            line = src.readline().rstrip()  # readline and remove trailing whitespace characters
            if line == "": # end of input reach, jump out of loop
                break
            elif match(r"^#.*$", line): # comments are ignored
                continue
            elif match(r"^init( \d+ \d+)+$", line): # init
                if not init:
                    raise ParseError(f"The keyword init occured after one out of ry, rz, cz, ccz, move")
                for x, y in get_coords(line):
                    if x not in pos2atoms: pos2atoms[x] = {}
                    pos2atoms[x][y] = atoms_n
                    atoms2pos.append((x, y))
                    csv.write(f"{lineno};{pos2atoms[x][y]};{x};{y};1;1\n")
                    atoms_n += 1
            else:
                init = False
                affected = set()
                try:
                    if match(r"^ry\(.+\)( \d+ \d+)+$", line): # ry (color: 2, size: 2)
                        for x, y in get_coords(line):
                            affected.add(pos2atoms[x][y])
                            csv.write(f"{lineno};{pos2atoms[x][y]};{x};{y};2;2\n")
                    elif match(r"^rz\(.+\)( \d+ \d+)+$", line): # rz (color: 3, size: 2)
                        for x, y in get_coords(line):
                            affected.add(pos2atoms[x][y])
                            csv.write(f"{lineno};{pos2atoms[x][y]};{x};{y};3;2\n")
                    elif match(r"^cz( \d+ \d+)+$", line): # cz (color: 4, size: 3)
                        for x, y in get_coords(line):
                            affected.add(pos2atoms[x][y])
                            csv.write(f"{lineno};{pos2atoms[x][y]};{x};{y};4;3\n")
                    elif match(r"^move( \d+ \d+ \d+ \d+)+$", line): # move
                        coords = [tuple(map(int, findall("\d+", p))) for p in findall(r"\d+ \d+ \d+ \d+", line)]
                        for x_old, y_old, x_new, y_new in coords:
                            affected.add(pos2atoms[x_old][y_old])
                            csv.write(f"{lineno};{pos2atoms[x_old][y_old]};{x_new};{y_new};1;1\n")
                        # update the both data-structures that store the locations of the atoms
                        pos2atoms_tmp = deepcopy(pos2atoms)
                        for x_old, y_old, x_new, y_new in coords:
                            # delete the old olcation
                            del pos2atoms[x_old][y_old]
                            # take the id from the deep-copy and store in new location
                            if x_new not in pos2atoms: pos2atoms[x_new] = {}
                            pos2atoms[x_new][y_new] = pos2atoms_tmp[x_old][y_old]
                            # update postion of atom
                            atoms2pos[pos2atoms_tmp[x_old][y_old]] = (x_new, y_new)
                        del pos2atoms_tmp
                    else:
                        raise ParseError(f"There is a syntax error in: {line}")
                    # write out all atoms with the last location that have not been affected by an operation above
                    for i in set(range(atoms_n)).difference(affected):
                        x, y = atoms2pos[i]
                        csv.write(f"{lineno};{i};{x};{y};1;1\n")
                except KeyError as e:
                    raise ParseError(f"The key {e} was not found in line {lineno}: {line}")
            lineno += 1


## Build Visualization

The above generated CSV file is taken and turned into a visulaization showing the movements and gate applications.

### Input

See the description of the output [above](#output).

### Output

A graphic showing the possible atom sites (SLM) in the background and the actual atoms including there movements. This figure can be exported in various formats using the functionality of plotly.

In [6]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

In [7]:
# Read in the two csv files generated above
df_atoms = pd.read_csv("data.csv", sep=";")
df_atoms.sort_values(by=['time_frame','atom'], inplace=True)
df_sites = pd.read_csv("sites.csv", sep=";")

In [8]:
# we use the library plotly.express to define the graphcics
# Firat, we define a graphics that shows the atoms as dots and animates there movement
atoms = px.scatter(df_atoms,
        height=800,
        width=750,
        x="xpos",
        y="ypos",
        animation_frame="time_frame",
        animation_group="atom",
        size="size",
        color="gate",
        hover_name="atom",
        size_max=10,
        range_x=[-6,120],
        range_y=[160,-6],
        range_color=[1,4],
        template="plotly_white")
atoms.layout.updatemenus[0].buttons[0].args[1]["frame"]["duration"] = 1000 # slow down animation
atoms.update(layout_coloraxis_showscale=False); # hide colorscale
# Second, we define a second graphic that will be used as the background showing the atom sites
sites = px.scatter(df_sites,
        x = "xpos",
        y = "ypos",
        size = "size",
        color = "zone",
        color_discrete_sequence = ["#bbf", "#bbb", "#fbb"],
        size_max = 5,
        range_x = [-6,120],
        range_y = [160,-6],
        template="plotly_white")

In [9]:
# Here, we combine the two figures above into one figure
# [SPOILER]: The following is very hacky, be warned!
# For an explanation: in principle two figures can be combined by invoking the method `add_traces` on one graphic and
# passing the content of the `.data` attribute of the other figure. However, in this case this results in either having
# the grid in the foreground or loosing the animation capability. Hence, we convert both figures to python dictionaries
# (to bypass integrity checks of the python objects).
#   1. We add the layers (one for every zone) showing the grid BEFORE the animation layer (`add_traces` would append them
#      to the end), and
#   2. Add three empty layers to each frame such that the atom layer is always the fourth one.
fig_dict = atoms.to_dict()
fig_dict["data"] = sites.to_dict()["data"] + fig_dict["data"]
for frame in fig_dict["frames"]:
    frame["data"] = [{},{},{},frame["data"][0]]
# Read-in the dictionary and display figure
fig = go.Figure(fig_dict)
fig.show()

In [14]:
with open("animation.html", "w") as file:
    file.writelines(fig.to_html(auto_play=False))